# AAPL Market Simulator

This notebook provides a step-by-step analysis of AAPL price action using an adaptive market simulator.

## Import Libraries

Let's start by importing the necessary libraries.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from market_model import MarketSimulator, Trader, OrderBook
from market_analysis import analyze_data, evaluate_predictions, plot_simulation_results

## Load Data

First, let's load the AAPL trading data.

In [2]:
def load_data(filepath):
    """Load AAPL trading data from CSV"""
    df = pd.read_csv(filepath, header=None, names=['timestamp', 'open', 'high', 'low', 'close', 'volume'],
                    parse_dates=['timestamp'])
    return df

# Load the data
df = load_data('AAPL_2024.csv')
print(f"Loaded {len(df)} records")
print("\nFirst 5 records:")
display(df.head())

## Data Analysis

Let's perform some initial analysis of the data.

In [3]:
# Perform initial analysis
analysis_results = analyze_data(df)

# Display key statistics
print("Key Statistics:")
for key, value in analysis_results.items():
    print(f"{key.replace('_', ' ').title()}: {value:.2f}")

# Display plots
from IPython.display import Image
print("\nPrice Trends:")
display(Image(filename='price_trends.png'))
print("\nVolume Trends:")
display(Image(filename='volume_trends.png'))

## Initialize Market Simulator

Now let's set up the market simulator with traders and order books.

In [4]:
# Initialize market components
order_book = OrderBook()
traders = [Trader(order_book) for _ in range(50)]  # Create 50 traders
simulator = MarketSimulator(order_book, traders)

print(f"Initialized with {len(traders)} traders")
print(f"Initial parameters: trader_activity_rate={order_book.trader_activity_rate}, proportion_maker={order_book.proportion_maker}")

## Parameter Optimization

Let's optimize the market parameters to best explain historical price action.

In [5]:
# Run parameter optimization
print("Running parameter optimization...")
best_params = simulator.optimize_parameters(df)

print("Optimized Parameters:")
for key, value in best_params.items():
    print(f"{key.replace('_', ' ').title()}: {value:.3f}")

## Run Market Simulation

Now let's run the market simulation using the optimized parameters.

In [6]:
# Run simulation with best parameters
print("Running market simulation...")
predictions = simulator.run_simulation(df, best_params)

# Evaluate results
rmse = evaluate_predictions(df['close'].values, predictions)
print(f"\nRoot Mean Squared Error (RMSE): {rmse:.4f}")

# Plot simulation results
errors = plot_simulation_results(df['close'].values, predictions)

# Display error statistics
print("\nError Statistics:")
print(f"Mean Error: {np.mean(errors):.4f}")
print(f"Std Error: {np.std(errors):.4f}")
print(f"Max Error: {np.max(errors):.4f}")
print(f"Min Error: {np.min(errors):.4f}")

## Analysis of Trader Behavior

Let's analyze how different types of traders behave in the simulation.

In [7]:
# Analyze trader behavior
print("Analyzing trader behavior...")

# Track orders placed by each trader type
maker_orders = []
taker_orders = []

# Simulate a short period to analyze behavior
sample_df = df.head(100)
order_book.last_traded_price = sample_df['open'].iloc[0]

for i in range(len(sample_df)):
    for trader in traders:
        order = trader.determine_price()
        if order['is_buy'] == (order['price'] < order_book.last_traded_price):
            # This is a maker order (buying low, selling high)
            maker_orders.append(order['price'])
        else:
            # This is a taker order (buying high, selling low)
            taker_orders.append(order['price'])
        
        # Update last traded price
        order_book.last_traded_price = sample_df['close'].iloc[i]

# Plot price distribution by trader type
plt.figure(figsize=(12, 6))
plt.hist(maker_orders, bins=30, alpha=0.5, label='Maker Orders', color='green')
plt.hist(taker_orders, bins=30, alpha=0.5, label='Taker Orders', color='red')
plt.title('Trader Order Price Distribution')
plt.xlabel('Price')
plt.ylabel('Number of Orders')
plt.legend()
plt.savefig('trader_behavior.png')
plt.close()

print("Trader behavior analysis complete. See trader_behavior.png")
print(f"Maker orders: {len(maker_orders)}, Taker orders: {len(taker_orders)}")
print(f"Average maker order price: {np.mean(maker_orders):.2f}")
print(f"Average taker order price: {np.mean(taker_orders):.2f}")

## Sensitivity Analysis

Let's perform sensitivity analysis to understand how different parameters affect the simulation.

In [8]:
# Sensitivity analysis
print("Running sensitivity analysis...")

activity_rates = np.linspace(0.2, 1.8, 5)
maker_proportions = np.linspace(0.2, 0.8, 5)
rmse_results = np.zeros((len(activity_rates), len(maker_proportions)))

for i, activity_rate in enumerate(activity_rates):
    for j, proportion_maker in enumerate(maker_proportions):
        params = {
            'trader_activity_rate': activity_rate,
            'proportion_maker': proportion_maker
        }
        predictions = simulator.run_simulation(df.head(200), params)
        rmse_results[i, j] = evaluate_predictions(df.head(200)['close'].values, predictions)

# Plot sensitivity results
plt.figure(figsize=(10, 8))
sns.heatmap(rmse_results, annot=True, fmt=".2f", cmap='viridis',
                xticklabels=[f"{p:.1f}" for p in maker_proportions],
                yticklabels=[f"{r:.1f}" for r in activity_rates])
plt.title('Sensitivity Analysis: RMSE by Parameters')
plt.xlabel('Proportion Maker')
plt.ylabel('Trader Activity Rate')
plt.savefig('sensitivity_analysis.png')
plt.close()

print("Sensitivity analysis complete. See sensitivity_analysis.png")
print(f"Best parameters from analysis: activity_rate={activity_rates[np.unravel_index(np.argmin(rmse_results), rmse_results.shape)[0]]:.2f}, "
      f"maker_proportion={maker_proportions[np.unravel_index(np.argmin(rmse_results), rmse_results.shape)[1]]:.2f}")

## Summary

Let's summarize our findings from the market simulation.

In [9]:
print("=== Market Simulation Summary ===")
print(f"Data period: {df['timestamp'].min()} to {df['timestamp'].max()}")
print(f"Total records: {len(df)}")
print(f"\nOptimized Parameters:")
print(f"  Trader Activity Rate: {best_params['trader_activity_rate']:.3f}")
print(f"  Proportion Maker: {best_params['proportion_maker']:.3f}")
print(f"\nSimulation Results:")
print(f"  RMSE: {rmse:.4f}")
print(f"  Mean Error: {np.mean(errors):.4f}")
print(f"  Std Error: {np.std(errors):.4f}")

print("\nKey Insights:")
print(f"  - The optimized trader activity rate suggests {best_params['trader_activity_rate']:.1f}x the base activity level")
print(f"  - Maker orders constitute {best_params['proportion_maker']*100:.0f}% of total orders")
print(f"  - The simulation achieves an RMSE of {rmse:.4f}, indicating moderate predictive accuracy")

print("\nVisualizations generated:")
print("  - price_trends.png: Historical price trends with moving averages")
print("  - volume_trends.png: Trading volume over time")
print("  - simulation_results.png: Actual vs predicted prices")
print("  - error_distribution.png: Distribution of prediction errors")
print("  - trader_behavior.png: Price distribution of maker vs taker orders")
print("  - sensitivity_analysis.png: RMSE sensitivity to parameters")